## 1.17 Example: Trades Table  

In this section we construct a toy trades table to demonstrate the power of q-sql. 

￼A useful operator for constructing lists of test data is (?), which generates pseudo-random data. We can generate 10 numbers randomly selected, with replacement, from the first 20 integers starting at 0 (i.e., not including 20). 

In [1]:
10?20 / ymmv 

12 8 10 1 9 11 5 6 1 5


In [2]:
10?20 = 10?20

0000000000b


We can similarly generate 10 random floats between 0.0 and 100.0 (not including 100.0). 

In [3]:
10?100.0

19.46509 9.059026 62.03014 93.26316 27.47066 5.752516 25.60658 23.10108 8.724..


In [6]:
/We can make 10 random selections from the items in a list 
10?(`one;`two)
10?`one`1

`two`one`two`one`one`two`two`one`two`one


`1`one`one`one`1`one`1`one`one`1



Now to our trades table. Since a table is a collection of columns, we first build the columns. We apologize for using excessively short names so that things fit easily on the printed page. 
First we construct a list of 1,000,000 random dates in the month of January 2015. 

In [7]:
dts:2015.01.01+1000000?31

In [8]:
dts

2015.01.22 2015.01.03 2015.01.28 2015.01.31 2015.01.20 2015.01.29 2015.01.13 ..


In [9]:
tms:1000000?24:00:00.000000000

In [10]:
/Next a list of 1,000,000 tickers chosen from AAPL, GOOG and IBM. It is customary to make these lower case symbols. 

In [11]:
syms:1000000?`aapl`goog`ibm

In [12]:
/Next a list of 1,000,000 volumes given as positive lots of 10
vols:10*1+1000000?1000

As an initial cut, we construct a list of 1,000,000 prices in cents uniformly distributed within 10% of 100.0. We will adjust this later. 

In [14]:
pxs:90.0+(1000000?2001)%100

In [13]:
/Now collect these into a table and inspect the first 5 records. Remember, a table is a list of records so (#) applies.

In [15]:
trades:([] dt:dts; tm:tms; sym:syms; vol:vols; px:pxs) 

In [16]:
5#trades

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.22 0D04:51:10.908354967 goog 4080 91.45 
2015.01.03 0D18:51:07.332527786 ibm  7180 94.26 
2015.01.28 0D23:10:38.237247616 aapl 1000 102.94
2015.01.31 0D21:43:55.479589551 aapl 8200 92.56 
2015.01.20 0D01:26:24.385518729 ibm  3650 92.4  


The first thing you observe in your console display is that the trades are not in temporal order. We fix this by sorting on time within date using (xasc). 

In [17]:
trades:`dt`tm xasc trades 

In [18]:
5#trades

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.01 0D00:00:07.004971057 goog 3240 93.67 
2015.01.01 0D00:00:09.415197372 goog 910  90.8  
2015.01.01 0D00:00:13.514088839 ibm  8180 107.25
2015.01.01 0D00:00:14.056894183 ibm  1190 94.72 
2015.01.01 0D00:00:15.205892175 aapl 8940 97.69 


Now we adjust the prices. At the time of this writing (Sep 2015) AAPL was trading around 100, so we leave it alone. But we adjust GOOG and IBM to their approximate trading ranges by scaling. 

In [19]:
trades:update px:6*px from trades where sym=`goog

In [20]:
5#trades

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.01 0D00:00:07.004971057 goog 3240 562.02
2015.01.01 0D00:00:09.415197372 goog 910  544.8 
2015.01.01 0D00:00:13.514088839 ibm  8180 107.25
2015.01.01 0D00:00:14.056894183 ibm  1190 94.72 
2015.01.01 0D00:00:15.205892175 aapl 8940 97.69 


In [21]:
trades:update px:2*px from trades where sym=`ibm

In [22]:
5#trades 

dt         tm                   sym  vol  px    
------------------------------------------------
2015.01.01 0D00:00:07.004971057 goog 3240 562.02
2015.01.01 0D00:00:09.415197372 goog 910  544.8 
2015.01.01 0D00:00:13.514088839 ibm  8180 214.5 
2015.01.01 0D00:00:14.056894183 ibm  1190 189.44
2015.01.01 0D00:00:15.205892175 aapl 8940 97.69 


This looks a bit more like real trades. Let’s perform some basic queries as sanity checks. Given that both price and volume are uniformly distributed, we expect their averages to approximate the mean. Using the built-in average function (avg) we see that they do. 

In [23]:
select avg px, avg vol by sym from trades 

sym | px       vol     
----| -----------------
aapl| 99.9981  5009.882
goog| 600.0105 5004.941
ibm | 200.024  5005.35 


Similarly, we expect the minimum and maximum price for each symbol to be the endpoints of the uniform range.

In [24]:
select min px, max px by sym from trades 

sym | px  px1
----| -------
aapl| 90  110
goog| 540 660
ibm | 180 220
